# Model

## Import libraries

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
import Functions_Books as funct
from fuzzywuzzy import process

c:\users\anton\appdata\local\programs\python\python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Import dataframe and define user input

In [2]:
df2=pd.read_csv('DataFinal\DataPreprocessed.csv',index_col=0)
df2.head()
#df.loc[4156,:]

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized,combined_features
0,Mistress Suffragette,none,Diana Forbes,1946409073,"Fiction, Novels, Drama, Contemporary, Historic...",A young woman without prospects at a ball in G...,333,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,12973,4.32,mistress suffragette,young woman without prospects ball gilded age ...,,mistress suffragette young woman without prosp...
1,Stones (One True Child #5),none,L.C. Conn,0,"Book Club, Novels, Fiction, Drama, Contemporary","Deep within the New Zealand bush, lies a plate...",327,2019,English,https://i.gr-assets.com/images/S/compressed.ph...,569,4.31,stones one true child 5,deep within new zealand bush lies plateau clea...,,stones one true child 5 deep within new zealan...
2,Mission: Subhero,none,Linda Armstrong,0,"Contemporary, Novels, Fiction, Drama, Book Clu...","With just a few lines of code, one average man...",225,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,2417,4.35,mission subhero,lines code one average man change world and se...,,mission subhero lines code one average man cha...
3,A Summer of Good-Byes (Blue Triangle Press Boo...,none,Fred Misurella,0,"Fiction, Novels, Adult, Contemporary, Adventur...",PASSION IN PROVENCE:Ben and Lee Alto follow Va...,0,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,1713,4.36,summer good byes blue triangle press book 2,passion provence ben lee alto follow van goghs...,,summer good byes blue triangle press book 2 pa...
4,The Scopas Factor,none,Vincent Panettiere,9781543947717,"Fiction, Contemporary, Novels, Drama, Book Clu...","A Hmong ""story cloth,"" a Revolutionary War bat...",326,2018,English,https://i.gr-assets.com/images/S/compressed.ph...,1080,4.31,scopas factor,hmong story cloth revolutionary war battle fla...,,scopas factor hmong story cloth revolutionary ...


In [3]:
title_user = input()

Harry Potter


## Suggest other titles based on user input - using Fuzzywuzzy

In [4]:
def suggestions_other_title_ForJupyter(title_user,df1):
    choices = df1['title'] 
    # Get a list of matches ordered by score, default limit to 5
    rec = process.extract(title_user, choices)
    str1="This exact title was not found in the database. Did you mean one of these?"
    #print()
    for i in rec:
        str1 += "\n"
        str1 +=i[0]
    suggestions = "This exact title was not found in the database. Did you mean one of these?\n", str(str1)
    x = str(str1)
    return print(str1)



In [5]:
def suggestions_other_title_ForStreamlit(title_user, df1):
    choices = df1['title'] 
    # Get a list of matches ordered by score, default limit to 5
    rec = process.extract(title_user, choices)
    #str1="This exact title was not found in the database. Did you mean one of these?"
    #print()
    j=0
    dictionary1={}
    for i in rec:
        dictionary1["str{0}".format(i)] = str(j+1)+' ' + i[0] 
        #str1 += "\n"
        #str1 +=i[0]
        j=j+1
    a=list(dictionary1.values())[:5]
    #suggestions = "This exact title was not found in the database. Did you mean one of these?\n", str(str1)
    #x = str(str1)
    return "This exact title was not found in the database. Did you mean one of these?",a[0],a[1],a[2],a[3],a[4],''

## Give 5 recommendations to a book title

In [6]:
#Need to define the recommendation Functions which follows later:
def give_5bookrecommendationsCount_forJupyter(title_user,df1):
    #1.Step - convert the description from combined_features to a matrix of word counts
    cm = CountVectorizer().fit_transform(df1['combined_features'])
    #2.Get the cosine_simmilarity from the count matrix
    cs = cosine_similarity(cm)
    #Get index of title of user input
    indices = df1[df1.title == title_user].index.values[0]
    #Create a list of tuples in the form (index,similarity)
    scores = list(enumerate(cs[indices]))
    # Sort the list of similar books in descending order
    sorted_score = sorted(scores, key=lambda x:x[1], reverse=True)
    sorted_score = sorted_score[1:]
    # Create a loop to print the first 5 books from the sorted list
    j=0
    #print('The 5 most recommended books to '+title_user+' are:\n')
    str2=''
    for item in sorted_score:
        book_title = df1[df1.index == item[0]]['title'].values[0]
        str2+=(str(j+1)+' ' + book_title + "\n")
        #print(j+1, book_title)
        j=j+1
        if j >=5:
            ert='The 5 most recommended books to '+str(title_user)+' are:\n' + str(str2)
            return print(ert)
        
#After explode do list and append only if title is not yet in list, if len(list) = 5 dann return

In [7]:
def give_5bookrecommendationsCount_ForStreamlit(title_user, df1):
    #1.Step - convert the description from combined_features to a matrix of word counts
    cm = CountVectorizer().fit_transform(df1['combined_features'])
    #2.Get the cosine_simmilarity from the count matrix
    cs = cosine_similarity(cm)
    #Get index of title of user input
    indices = df1[df1.title == title_user].index.values[0]
    #Create a list of tuples in the form (index,similarity)
    scores = list(enumerate(cs[indices]))
    # Sort the list of similar books in descending order
    sorted_score = sorted(scores, key=lambda x:x[1], reverse=True)
    sorted_score = sorted_score[1:]
    # Create a loop to print the first 5 books from the sorted list
    j=0
    #print('The 5 most recommended books to '+title_user+' are:\n')
    d={}
    for item in sorted_score:
        book_title = df1[df1.index == item[0]]['title'].values[0]
        d["str{0}".format(item)] = str(j+1)+' ' + book_title 
        #str{item}.append(str(j+1)+' ' + book_title + "\n")
        #print(j+1, book_title)
        j=j+1
    f=list(d.values())[:5]
       # if j >=5:
    return 'The 5 most recommended books to '+ str(title_user)+' are:', f[0],f[1],f[2],f[3],f[4],''

## Assume title based on input (fuzzy ration > 95)

In [8]:
def assume_book_title_ForJupyter(title_user,df1):
    choices = df1['title'] 
    # Get a list of matches ordered by score, default limit to 5
    rec = process.extract(title_user, choices)
    lst=[]
    for i in rec:
        lst.append(i[1])
        #print(i[1])
        if lst[0]>95:
            title_user=i[0]#list.sort(key=lst, reverse=True)
            return print('I did not find an exact match in the database. I assume you mean the book: ' + title_user), give_5bookrecommendationsCount_forJupyter(title_user,df1)
        else:
            error

In [9]:
def assume_book_title_ForStreamlit(title_user,df1):
    choices = df1['title'] 
    # Get a list of matches ordered by score, default limit to 5
    rec = process.extract(title_user, choices)
    lst=[]
    for i in rec:
        lst.append(i[1])
        #print(i[1])
        if lst[0]>95:
            title_user=i[0]#list.sort(key=lst, reverse=True)
            return 'I did not find an exact match in the database. I assume you mean the book: '+ title_user, give_5bookrecommendationsCount_ForStreamlit(title_user, df1)[0],give_5bookrecommendationsCount_ForStreamlit(title_user, df1)[1],give_5bookrecommendationsCount_ForStreamlit(title_user, df1)[2],give_5bookrecommendationsCount_ForStreamlit(title_user, df1)[3],give_5bookrecommendationsCount_ForStreamlit(title_user, df1)[4],give_5bookrecommendationsCount_ForStreamlit(title_user, df1)[5]
        else:
            error

## Combine all functions - Complete function

In [10]:
def give_recommendationComplete_ForJupyter(title_user,df1):
    try:
        #print(1)
        a = give_5bookrecommendationsCount_ForJupyter(title_user,df1)
        if a:
            return a
    except:
        try:
            #print(2)
            b = assume_book_title_ForJupyter(title_user,df1)
            if b:
                return b 
        except:
            #print(3)
            c= suggestions_other_title_ForJupyter(title_user,df1) 
            if c:
                return c
    

In [11]:
def give_recommendationComplete_ForStreamlit(title_user,df1):
    try:
        #print(1)
        a = give_5bookrecommendationsCount_ForStreamlit(title_user,df1)
        if a:
            return a
    except:
        try:
            #print(2)
            b = assume_book_title_ForStreamlit(title_user,df1)
            if b:
                return b 
        except:
            #print(3)
            c= suggestions_other_title_ForStreamlit(title_user,df1) 
            if c:
                return c